# Предобработка исходных данных с отзывами

!!! Осторожно, BigData

In [1]:
import pandas as pd
import numpy as np
import string

from sklearn.model_selection import train_test_split

RANDOM_STATE = 2

## Описание исходных данных

In [2]:
reviews = pd.read_csv('reviews.csv')
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,9554,1184025,2012-04-26,1809049,Hana,"I stayed in London for a month to study, exp..."
1,9554,1206322,2012-04-30,2237488,Rishi,My bnb request was very last minute and i was ...
2,9554,1258541,2012-05-10,2150467,Panee,First time as airbnb!First time to London! I c...
3,9554,1405284,2012-06-03,1864672,Simone Cristina,A wonderful experience! The house is very well...
4,9554,1475969,2012-06-13,2438453,Sondra,We are so grateful that we trusted our intuiti...
...,...,...,...,...,...,...
1137319,29735949,344807582,2018-11-04,27552372,Declan,"Clean, spacious, stylish apartment close to ev..."
1137320,29736900,344387254,2018-11-03,109537206,Shazia,spotless clean flat with amazing view. the hos...
1137321,29756033,344498174,2018-11-03,26000990,Andrea,The host canceled this reservation 55 days bef...
1137322,29775194,345574439,2018-11-05,34546792,Robert,The host canceled this reservation 45 days bef...


### Схема данных

Поля 'id', 'date', 'reviewer_id', 'reviewer_name' содержат метаданные отзыва, \
мы не будем их использовать.

Поля 'listing_id' и 'comments' содержат идентификатор аренды и текст отзыва, \
будем использовать их для построения модели.

### Статистика пропусков и уникальных отзывов

In [3]:
reviews[['listing_id','comments']] \
    .describe(include='all') \
    .T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
listing_id,1.13732e+06,NaN,NaN,NaN,1.17062e+07,8.04298e+06,9554,4.2822e+06,1.17502e+07,1.83234e+07,2.97936e+07
comments,1135996,1095139,The host canceled this reservation the day bef...,1519,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Статистика отзывов по идентификатору аренды

In [4]:
reviews[['listing_id','comments']].dropna() \
    .groupby(by=['listing_id']) \
    .count() \
    .describe() \
    .T

,count,mean,std,min,25%,50%,75%,max
comments,56278.0,20.185437,35.357456,1.0,3.0,8.0,22.0,566.0


### Статистика отзывов по длине текста

In [5]:
reviews.comments.dropna().apply(lambda x: len(x)).to_frame().describe().T

,count,mean,std,min,25%,50%,75%,max
comments,1135996.0,278.719857,262.380617,1.0,105.0,209.0,365.0,6743.0


### Статистика отзывов по количеству слов
Количество слов аппроксимируется количеством пробелов в тексте

In [6]:
reviews.comments.dropna().apply(lambda x: len(x) - len(x.replace(' ', ''))).to_frame().describe().T

,count,mean,std,min,25%,50%,75%,max
comments,1135996.0,48.506511,47.653814,0.0,17.0,36.0,64.0,1008.0


### Алфавит и язык отзывов

In [7]:
reviews.comments.dropna().sort_values()

87743     \t\thello \r\ntraveled July/12 – July/20 we wh...
658497    \t\n\nKamey is a great host and friendly guy, ...
65515     \tMe, my wife and granddaughter lived in this ...
502662                                                   \n
544602                                                   \n
                                ...                        
627670    ﾛﾝﾄﾞﾝｼﾃｨ空港を利用するため2泊で利用しました｡\nｽﾀｯﾌの方は優しく､たどたどしい...
378810    ﾛﾝﾄﾞﾝｾﾝﾄﾗﾙからのｱｸｾｽもよく､観光にも非常に便利だった｡\r\nﾎｽﾄのﾏｷﾞｰ...
135693    ﾛﾝﾄﾞﾝﾌｨｰﾙｽﾞに近く､その周りを囲むように美味しいﾍﾞｰｶﾘｰやﾛｰｽﾀﾘｰｶﾌｪな...
487165    ﾛﾝﾄﾞﾝﾌﾞﾘｯｼﾞからの連絡もよく､ｹﾋﾞﾅさんもとてもいい人です｡女2人で寝るには大き...
961470    ﾜﾝちゃんがなついてくれて楽しかったです｡深夜のﾁｪｯｸｲﾝにも対応してくれました｡駅から少...
Name: comments, Length: 1135996, dtype: object

### Обобщенное описание данных

Датасет содержит 1137324 записи, \
из них с непустыми отзывами 1135996, \
уникальных отзывов 1095139.

Объектов аренды: 56278. \
Отзывов по каждому объекту от 1 до 566, в среднем 20 отзывов. \
Более 25% объектов имеют 1 отзыв. \
Более 50% объектов имеют 3 и меньше отзывов. \
Более 75% объектов имеют 8 и меньше отзывов. 

Отзывы на разных языках, оставим только латиницу и цифры. 

Встречаются отзывы, содержащие один символ, содержащие одно слово и т.п. \
Предполагаем, что отзывы, содержащие менее двух слов являются не информативными.

Также часто встречаются отзывы об отмене брони: 'The host canceled this reservation ...' \
Оне тоже не предоставляют информацию для решения задачи.

## Предобработка данных

In [8]:
reviews.shape

(1137324, 6)

### Удаление пропусков

In [9]:
comments = reviews.loc[reviews.comments.notna(), ['listing_id','comments']].copy()
comments.shape

(1135996, 2)

### Очистка текста

In [10]:
punct = str([c for c in string.punctuation if not c in '@#$%&*+-_/'])
trans = str.maketrans(punct, ' ' * len(punct))

comments['comments'] = comments['comments'] \
    .str.lower() \
    .str.replace(r'(\n|\r|\t)', ' ', regex=True) \
    .str.replace(r'[^a-z0-9@#$%&*+-_/]+', ' ', regex=True) \
    .str.translate(trans) \
    .str.replace('\s+', ' ', regex=True) \
    .str.strip()
    
empty_word_mask = comments.comments.map(len) == 0
comments.drop(index=comments[empty_word_mask].index, axis=0, inplace=True)

comments.sort_values(by='comments')

,listing_id,comments
1056790,23820750,#1 - the most comfortable bed #2 - 50-minute e...
1096433,25861190,#1 pick for staying in london amazing location...
476085,8844647,#1 room to rent in london
652718,13903095,#10 osten mews is a wonderful and very central...
268896,3957718,#en# reinaldo is a very nice host we spent som...
...,...,...
920886,19895633,zwei sch ne terrassen sehr angenehme kommunika...
133063,1131354,zwei sehr nette zimmer optimal f r eine famili...
335967,5293527,zwei super liebe und hilfsbereite menschen mei...
70649,539042,zwei wochen war ich in london und bin sehr dan...


### Удаление отзывов, состоящих из малого количества слов (до 10)

In [11]:
one_word_mask = comments.comments.map(lambda x: len(x) - len(x.replace(' ', ''))) < 10
comments.drop(index=comments[one_word_mask].index, axis=0, inplace=True)
comments.shape

(989607, 2)

### Удаление отзывов, содержащих малое количество символов (до 30)

In [12]:
few_symbols_mask = comments.comments.apply(lambda x: len(x)) < 30
comments.drop(index=comments[few_symbols_mask].index, axis=0, inplace=True)
comments.shape

(989594, 2)

### Удаление отзывов с информацие об отмене аренды

In [13]:
host_cancelled_mask = comments.comments.apply(lambda x: x.startswith('the host canceled this reservation'))
comments.drop(index=comments[host_cancelled_mask].index, axis=0, inplace=True)

host_cancelled_mask = comments.comments.apply(lambda x: x.startswith('the reservation was canceled'))
comments.drop(index=comments[host_cancelled_mask].index, axis=0, inplace=True)

comments.shape

(967567, 2)

In [14]:
comments.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
listing_id,967567,NaN,NaN,NaN,1.12976e+07,8.00188e+06,9554,3.88846e+06,1.11139e+07,1.7899e+07,2.97369e+07
comments,967567,966936,a very nice comfortable stay for my group frie...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Результат предобработки исходных данных

In [15]:
comments.index.rename('id', inplace=True)
comments.sort_values(by='comments')

,listing_id,comments
id,,
1056790,23820750,#1 - the most comfortable bed #2 - 50-minute e...
1096433,25861190,#1 pick for staying in london amazing location...
652718,13903095,#10 osten mews is a wonderful and very central...
268896,3957718,#en# reinaldo is a very nice host we spent som...
1054528,23727374,#super cool location #clean #cool #design flat...
...,...,...
920886,19895633,zwei sch ne terrassen sehr angenehme kommunika...
133063,1131354,zwei sehr nette zimmer optimal f r eine famili...
335967,5293527,zwei super liebe und hilfsbereite menschen mei...


In [16]:
comments.shape

(967567, 2)

Получился достаточно большой датасет, возьмем часть

In [17]:
comments_train, _, _, _ = train_test_split(comments, comments.listing_id,
                                     train_size=0.4,
                                     random_state=RANDOM_STATE,
                                     shuffle=True)
comments_train.shape

(387026, 2)

Сохраним полученные данные

In [18]:
comments_train.to_csv('comments.csv')
'Done'

'Done'